In [1]:
import matplotlib.pyplot as plt
import numpy as np

from neuropacks import HC6
from neuropacks import NHP
from pyuoi.decomposition import CUR
from scipy.io import loadmat
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [2]:
nhp = NHP(data_path='/Users/psachdeva/data/NHP/indy_20160411_01.mat')

In [2]:
hc6 = HC6('/Users/psachdeva/data/hc6/Bon')

In [35]:
day = 6
epoch = 0
bin_width = 0.5
max_k = 20

In [37]:
x_binned, y_binned = hc6.get_binned_positions(
    day=day,
    epoch=epoch,
    bin_width=bin_width
)

In [33]:
#Y = hc6.get_response_matrix(
#    day=day,
#    epoch=epoch,
#    bin_width=bin_width,
#    transform=None,
#    clean=True
#)
Y = nhp.get_response_matrix(
    bin_width=0.25, region='M1'
)
cursor_pos = nhp.get_binned_positions(bin_width=0.25)

In [136]:
max_k = 40
cur = CUR(
    n_boots=20,
    max_k=max_k,
    boots_frac=0.8
)
cur.fit(Y)

CUR(algorithm='randomized', boots_frac=0.8, max_k=40, n_boots=20, n_iter=5,
  random_state=None, tol=0.0)

In [142]:
print(cur.columns_)
cur.columns_

[  0  38  41  42  45  54  68  74  86 104 111 132 169 171 174 176]


array([  0,  38,  41,  42,  45,  54,  68,  74,  86, 104, 111, 132, 169,
       171, 174, 176])

In [138]:
_, _, V = np.linalg.svd(Y)

cur_flags = CUR.column_select(V[:max_k].T, max_k + 20)
n_cur = np.count_nonzero(cur_flags)

In [143]:
cur_flags

array([ True, False,  True,  True, False, False, False,  True, False,
        True, False, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False,  True, False,  True,  True,  True,  True,  True,
        True, False,  True,  True, False, False,  True, False,  True,
        True, False, False, False, False, False, False, False, False,
       False,  True, False, False, False,  True, False, False, False,
        True, False,  True, False, False, False,  True, False, False,
       False, False, False, False, False,  True, False,  True, False,
        True, False, False, False, False, False, False, False,  True,
        True, False, False, False, False,  True, False, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False, False,  True, False,  True,  True, False,
        True, False,

In [139]:
print(np.argwhere(cur_flags).ravel())
print(np.count_nonzero(cur_flags))

[  0   2   3   7   9  15  18  30  38  40  41  42  43  44  45  47  48  51
  53  54  64  68  72  74  78  86  88  90  98  99 104 108 111 114 117 121
 123 124 126 129 131 132 133 134 137 139 145 146 147 149 151 159 161 165
 169 170 171 174 175 176 178 186 187]
63


In [140]:
Yc_uoi = Y[:, cur.columns_]
Yc_cur = Y[:, cur_flags]

uoidel = np.dot(Yc_uoi, np.dot(np.linalg.pinv(Yc_uoi), Y)) - Y
curdel = np.dot(Yc_cur, np.dot(np.linalg.pinv(Yc_cur), Y)) - Y

In [141]:
print(np.count_nonzero(uoidel)/Y.size)
print(np.count_nonzero(curdel)/Y.size)

0.7587925627240143
0.7496243662033395


In [28]:
ols = LinearRegression()
ols.fit(Y[:1500, cur.columns_], cursor_pos[:1500, 1])
print(ols.score(Y[1500:, cur.columns_], cursor_pos[1500:, 1]))
ols.fit(Y[:1500, cur_flags], cursor_pos[:1500, 1])
print(ols.score(Y[1500:, cur_flags], cursor_pos[1500:, 1]))

0.2084368411582912
0.3665473487370742


In [13]:
ols = LinearRegression()
ols.fit(Y[:1500][:, cur_flags], x_binned[:1500])
print(ols.score(Y[1500:][:, cur_flags], x_binned[1500:]))
ols.fit(Y[:1500][:, cur_flags], y_binned[:1500])
print(ols.score(Y[1500:][:, cur_flags], y_binned[1500:]))

NameError: name 'x_binned' is not defined

In [20]:
for neuron in range(Y.shape[1]):
    ols = LinearRegression()
    ols.fit(Y[:1500][:, cur.columns_], Y[:1500, neuron])
    print(ols.score(Y[1500:][:, cur.columns_], Y[1500:, neuron]))
    
    ols = LinearRegression()
    ols.fit(Y[:1500][:, cur_flags], Y[:1500, neuron])
    print(ols.score(Y[1500:][:, cur_flags], Y[1500:, neuron]))

0.14755462665601282
1.0
0.09731538446472708
1.0
0.04711971285661831
1.0
-0.02002949294888756
1.0
0.060063988768590426
0.03719515284273245
0.28785734999705137
0.31786222592362423
-0.029207851610281832
-0.08697692347965136
0.09237853379032301
0.1534045778681571
0.0
0.0
0.16108728694111796
0.21645739271193676
0.07555961813132273
1.0
1.0
1.0
0.00398798887839269
-0.07328563519974662
0.21496480932037887
0.2939607125687812
0.12341482829760388
0.188391112880252
0.23374954048091956
1.0
-0.018366789490949387
-0.03307343589490386
-0.0010054321506069996
-0.013077935405134022
0.16308300955158161
1.0
1.0
1.0
0.13370259267125884
0.15105209117034235
-0.03080002505789059
1.0
0.33060581428140356
0.4296535335622773
0.25985095480368514
1.0
0.017590708282727796
1.0
0.06432507001711807
0.1295993124984488
-0.03278241841869689
-0.1422866652527477
1.0
1.0
1.0
1.0
0.27134309493705255
0.38207797254734055
1.0
1.0
0.3049731531447788
0.4361441070534028
1.0
1.0
1.0
1.0
0.0430585609729297
1.0
0.036691522883204875
0.0

In [16]:
for neuron in range(Y.shape[1]):
    ols = LinearRegression()
    ols.fit(Y[:1500][:, cur_flags], Y[:1500, neuron])
    print(ols.score(Y[1500:][:, cur_flags], Y[1500:, neuron]))

1.0
1.0
1.0
1.0
0.03719515284273245
0.31786222592362423
-0.08697692347965136
0.1534045778681571
0.0
0.21645739271193676
1.0
1.0
-0.07328563519974662
0.2939607125687812
0.188391112880252
1.0
-0.03307343589490386
-0.013077935405134022
1.0
1.0
0.15105209117034235
1.0
0.4296535335622773
1.0
1.0
0.1295993124984488
-0.1422866652527477
1.0
1.0
0.38207797254734055
1.0
0.4361441070534028
1.0
1.0
1.0
0.0606326700291554
0.08196638585765381
0.016699331221184166
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-0.007425742574257655
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
-0.024788688215767966
1.0
1.0
0.15611838097027098
1.0
0.030998943815826352
0.007311729668689649
1.0
1.0
1.0
0.4009569176830784
1.0
1.0
1.0
0.41877632126878334
0.025724373955965604
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-0.011611230975675069
0.0681045221305655
0.0733898661244452
-0.02555391494915593
0.09318080037754195
0.12485677496283187
1.0
-0.05718389727979001
0.27488182417757334
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.004192092217733645
1.0
1